<a href="https://colab.research.google.com/github/abhinav70291/ML-system-design/blob/main/Abhinav_submission_for_matrice_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import psutil
import time
from torchvision.models.detection import *

# ----------------------------------------------------------------
# Here's a handy function to fetch a model based on its name
# ----------------------------------------------------------------

def get_model(model_name, pretrained=True, device='cpu'):
    model_dict = {
        'maskrcnn_resnet50_fpn': maskrcnn_resnet50_fpn,
        'fasterrcnn_resnet50_fpn': fasterrcnn_resnet50_fpn,
        'fcos_resnet50_fpn': fcos_resnet50_fpn,
        'retinanet_resnet50_fpn': retinanet_resnet50_fpn,
        'ssd300_vgg16': ssd300_vgg16,
        'ssdlite320_mobilenet_v3_large': ssdlite320_mobilenet_v3_large
    }

    if model_name in model_dict:
        model = model_dict[model_name](pretrained=pretrained)
        return model.to(device)
    else:
        raise ValueError(f"Unknown model: {model_name}")



# ----------------------------------------------------------------
# This function checks for a GPU, using a CPU as a backup
# ----------------------------------------------------------------

def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# ----------------------------------------------------------------
# Let's estimate how much memory a model might need
# ----------------------------------------------------------------
def get_model_memory_usage(batch_size, model):
    float_bytes = 4.0  # Multiplication factor as all values we store would be float32.
    features_mem = 0  # Initialize memory to store  features.

    for param in model.parameters():
        # here I am multiplying all shapes to get the total number per layer.
        single_layer_mem = np.prod(param.shape)
        single_layer_mem_float = single_layer_mem * float_bytes
        single_layer_mem_MB = single_layer_mem_float / (1024 ** 2)  # Converting to MB

        features_mem += single_layer_mem_MB  # Add to total feature memory count

    # Calculate Parameter memory
    trainable_wts = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_wts = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    parameter_mem_MB = ((trainable_wts + non_trainable_wts) * float_bytes) / (1024 ** 2)
    print("Memory for Model features in GB is:", features_mem * batch_size/1024, "GB")
    print("Memory for Model parameters in MB is: %.2f" % parameter_mem_MB)

    total_memory_MB = (batch_size * features_mem) + parameter_mem_MB
    total_memory_GB = total_memory_MB / 1024

    # Checking if a GPU is available and if so, get its details
    if torch.cuda.is_available():
        device = torch.device('cuda')
        print(torch.cuda.memory_summary(device=device, abbreviated=False))
        print("Device name:", torch.cuda.get_device_name(device))
        print("Memory Allocated:", torch.cuda.memory_allocated(device)/1024**3, "GB")
        print("Memory Cached:   ", torch.cuda.memory_reserved(device)/1024**3, "GB")


    # Get CPU and RAM usage
    cpu_usage = psutil.cpu_percent()
    ram_usage = psutil.virtual_memory().percent
    print("CPU usage: %.2f%%" % cpu_usage)
    print("RAM usage: %.2f%%" % ram_usage)

    return total_memory_GB

# ----------------------------------------------------------------
# Here's a way to estimate training and inference times
# ----------------------------------------------------------------
def estimate_time(device, batch_size, epochs, model, total_samples, input_size=(3, 224, 224)):

    with torch.no_grad():  # Disable gradient calculation globally
        # Create a random input tensor on the CPU
        input_tensor = torch.rand(batch_size, *input_size, device='cpu')

        # Move the input tensor to the GPU
        input_tensor_list = [input_tensor[i].to(device) for i in range(batch_size)]

        # Move the model to the GPU only for this estimation
        model.to(device)
        # Measure the time taken for a forward pass
        start_time = time.time()
        model(input_tensor_list)  # Pass a list of tensors to the model
        if device=="cuda":
            torch.cuda.synchronize()  # Ensure accurate timing
        end_time = time.time()

        inference_time = end_time - start_time

        # Calculate the number of batches per epoch
        batches_per_epoch = total_samples // batch_size

        # Estimate training time (approximately twice the inference time)
        training_time = 2 * inference_time * epochs * batches_per_epoch

        # Move the model back to the CPU
        model.to('cpu')

        # Explicitly clear GPU cache
        if device=="cuda":
            torch.cuda.empty_cache()

    return training_time, inference_time



device = get_device()
# Here are some models we're about to experiment
model_names = ['maskrcnn_resnet50_fpn', 'fasterrcnn_resnet50_fpn', 'fcos_resnet50_fpn', 'retinanet_resnet50_fpn', 'ssd300_vgg16', 'ssdlite320_mobilenet_v3_large']


# Some settings for our experiments
batch_size = 8
epochs = 3
total_samples = 50000  # Replace with your dataset size

# Let's loop through the models and see how they fare
for model_name in model_names:
    model = get_model(model_name, device=device)  # Load the model onto the right device
    model.eval()  # Set it to evaluation mode

    print(f"Model: {model_name}")
    print("Memory usage (GB):", get_model_memory_usage(batch_size, model))
    training_time, inference_time = estimate_time(device, batch_size, epochs, model, total_samples, input_size=(3, 800, 800))
    print("Estimated training time: %.2f seconds" % training_time)
    print("Estimated inference time: %.2f seconds" % inference_time)
    print("_________________________________________________________________________________")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:03<00:00, 58.5MB/s]


Model: maskrcnn_resnet50_fpn
Memory for Model features in GB is: 1.323264628648758 GB
Memory for Model parameters in MB is: 169.38
CPU usage: 39.70%
RAM usage: 10.70%
Memory usage (GB): 1.4886727072298527
Estimated training time: 2142803.31 seconds
Estimated inference time: 57.14 seconds
_________________________________________________________________________________


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 117MB/s]


Model: fasterrcnn_resnet50_fpn
Memory for Model features in GB is: 1.2444044947624207 GB
Memory for Model parameters in MB is: 159.28
CPU usage: 65.00%
RAM usage: 13.00%
Memory usage (GB): 1.3999550566077232
Estimated training time: 2102652.73 seconds
Estimated inference time: 56.07 seconds
_________________________________________________________________________________


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCOS_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FCOS_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fcos_resnet50_fpn_coco-99b0c9b7.pth" to /root/.cache/torch/hub/checkpoints/fcos_resnet50_fpn_coco-99b0c9b7.pth
100%|██████████| 124M/124M [00:01<00:00, 79.4MB/s]


...

Model: retinanet_resnet50_fpn
Memory for Model features in GB is: 1.0137259662151337 GB
Memory for Model parameters in MB is: 129.76
CPU usage: 62.30%
RAM usage: 12.70%
Memory usage (GB): 1.1404417119920254
Estimated training time: 2104735.31 seconds
Estimated inference time: 56.13 seconds
_________________________________________________________________________________


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to /root/.cache/torch/hub/checkpoints/ssd300_vgg16_coco-b556d3b4.pth
100%|██████████| 136M/136M [00:03<00:00, 39.1MB/s]


Model: ssd300_vgg16
Memory for Model features in GB is: 1.0622091889381409 GB
Memory for Model parameters in MB is: 135.96
CPU usage: 60.10%
RAM usage: 12.90%
Memory usage (GB): 1.1949853375554085
Estimated training time: 445537.09 seconds
Estimated inference time: 11.88 seconds
_________________________________________________________________________________


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SSDLite320_MobileNet_V3_Large_Weights.COCO_V1`. You can also use `weights=SSDLite320_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/ssdlite320_mobilenet_v3_large_coco-a79551df.pth" to /root/.cache/torch/hub/checkpoints/ssdlite320_mobilenet_v3_large_coco-a79551df.pth
100%|██████████| 13.4M/13.4M [00:00<00:00, 47.2MB/s]


Model: ssdlite320_mobilenet_v3_large
Memory for Model features in GB is: 0.10252177715301514 GB
Memory for Model parameters in MB is: 13.12
CPU usage: 59.70%
RAM usage: 12.80%
Memory usage (GB): 0.11533699929714203
Estimated training time: 31191.68 seconds
Estimated inference time: 0.83 seconds
_________________________________________________________________________________
